In [6]:
import os, sys
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error
import math
from datetime import datetime, timedelta

# Set path to current directory
#sys.path.append(os.path.abspath(os.getcwd()))

# Change as required
os.chdir('F:/Users/Dave/Desktop/UNSW-Capstone-Group-E/src/chrisdavid')

# Load data and split target and features

# Set this to the percentage of the full data file we want to use. Smaller percentages will reduce computation time for testing
#PROPORTION = 1

#Random state for repeatable runs
STATE = 42

data = pd.read_csv('./Cleaned_Data_mkII.csv')
#data = data[data['STATE'] == 'NSW'] # We are only looking at NSW here

#data = data.sample(frac = PROPORTION)

data['DATETIME'] = pd.to_datetime(data['DATETIME'])



In [7]:
#data['DATETIME'] = data['DATETIME'].dt.floor(freq='4H')
data = data.groupby('DATETIME').agg({'TEMPERATURE':'mean',
                                'TOTALDEMAND':'mean'}).reset_index()

In [8]:
weather = pd.read_csv('weather.csv')

weather['DATE'] = pd.to_datetime(weather[['Year','Month','Day']]).dt.date

weather.columns = ['drop1','drop2','drop3','drop4','drop5','rain','drop6','drop7','DAY']
weather = weather[['DAY','rain']]

weather['rain'].fillna(method='ffill', inplace=True)
weather['RAIN'] = pd.cut(weather['rain'],bins=[-1,0.2,4,10,999999],labels=['NONE','LIGHT','MODERATE','HEAVY'])

weather_dummy = pd.get_dummies(weather,columns=['RAIN'])






In [9]:
# Dummy coding for states
# Not needed because states not used

data['DAY'] = data['DATETIME'].dt.date
data['DAILY_MAX_TEMP'] = data.groupby('DAY')['TEMPERATURE'].transform('max')

data = data.merge(weather_dummy, on='DAY')




In [10]:
#def get_delay_temp(row, delay):
#    delay_date = row['DATE_SAVE'] - timedelta(hours=delay)
#    try:
#        return data.loc[delay_date]['TOTALDEMAND']
#    except KeyError:
#        return 0

#data['DATE_SAVE'] = data['DATETIME']
#data.set_index('DATETIME', inplace=True)

#data['DEMAND_4HOURDELAY'] = data.apply(get_delay_temp, delay=4, axis=1)
#data['DEMAND_8HOURDELAY'] = data.apply(get_delay_temp, delay=8, axis=1)
#data['DEMAND_24HOURDELAY'] = data.apply(get_delay_temp, delay=24, axis=1)
#data.reset_index(inplace=True)


features = data[['TEMPERATURE','DAILY_MAX_TEMP','RAIN_NONE','RAIN_LIGHT','RAIN_MODERATE','RAIN_HEAVY']]
target = data['TOTALDEMAND']



In [ ]:
features

In [11]:
# Convert to numpy arrays and split training/test data

features_np = pd.DataFrame(features).to_numpy()
target_np = np.ravel(pd.DataFrame(target).to_numpy())

features_train, features_test, target_train, target_test = train_test_split(features_np, target_np, random_state= STATE)

In [12]:
# Implement Random Forest
# TODO: Make it better

#Hyperparameter tuning grid

rnd_clf = RandomForestRegressor(random_state = STATE)
rnd_clf.fit(features_train, target_train)

rf_predicted = rnd_clf.predict(features_test)
#rf_diff = target_test - rf_predicted


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\fixes.py:357: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if _joblib.__version__ >= LooseVersion('0.12'):
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\base.py:158: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#dep

In [ ]:
rf_diff

In [13]:
# Implement Support Vector Machine Regressor
# TODO: Make it better

svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(features_train, target_train)
svm_predicted = svm_reg.predict(features_test)

In [14]:
# Calculate RMSE for each model and benchmark
# Lower = better model

rf_error = round(math.sqrt(mean_squared_error(target_test, rf_predicted)))
svm_error = round(math.sqrt(mean_squared_error(target_test, svm_predicted)))
#benchmark_error = round(math.sqrt(mean_squared_error(data['TOTALDEMAND'], data['FORECASTDEMAND'])))

print("Random Forest Error: ", rf_error)
print("SVM Error: ", svm_error)
#print("Benchmark Error: ", benchmark_error)

Random Forest Error:  1127
SVM Error:  1297
